<a href="https://colab.research.google.com/github/M-Crowson/2020EssexChallengeWeekGroup5/blob/Data-Cleansing-Functions/WebScrapingMachine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Tweepy
!pip install tweepy

In [22]:
import re
import tweepy
from tweepy import OAuthHandler
consumer_api_key = 'nCSS9AOFEtjC6z3hSrfdgh760'
consumer_api_secret = 'zj6ELUiIIycKmSEx6jzUgNIYBGLPk7uLnF241rGzmHvcgHQJ9H' 
access_token = '1315955315314167808-3rEY0WXk5XasWU1AgCEX6Ahbkjonzs'
access_token_secret ='rSc7JXXHqwbbqAHOp3qHUwSYpLC8ViXybnnCXs3M4DNWI'

In [23]:
authorizer = OAuthHandler(consumer_api_key, consumer_api_secret)
authorizer.set_access_token(access_token, access_token_secret)

In [24]:
api = tweepy.API(authorizer ,timeout=15)
all_tweets = []
search_query = '#AppleEvent'

for tweet_object in tweepy.Cursor(api.search,q=search_query+" -filter:retweets",lang='en',result_type='recent').items(200):
    #filter out tweets that don't have a place attached to them 
    if tweet_object.place != None:
      all_tweets.append({"text": tweet_object.text, "country": tweet_object.place.country, "country_code" : tweet_object.place.country_code})
      print("Text: " + tweet_object.text)
      print("Country: " + str(tweet_object.place.country))
      print("Country Code: " + str(tweet_object.place.country_code))
      print("--------------------------------------------------------------------------------")


Text: #AppleEvent
Basically Apple thinks even if they will sell poop people will buy it 🤷🏻‍♂️.

It's just a 4s with bette… https://t.co/EeAXgwSzCf
Country: Pakistan
Country Code: PK
--------------------------------------------------------------------------------
Text: #AppleEvent
I have joke on iphone 12 
But go and buy it for me first. 
Then I'll tell u.😏😉
@Apple #iPhone12
Country: India
Country Code: IN
--------------------------------------------------------------------------------
Text: I loved the blue 😍😍
#AppleEvent
#Apple12
#iPhone12
Country: Kingdom of Saudi Arabia
Country Code: SA
--------------------------------------------------------------------------------
Text: #AppleEvent
It's just iphone 4s with a better body but price 🤦🏼‍♂️🤔.

Without charger and earphones 🤦🏼‍♂️.

Beyond… https://t.co/zzHwUV6aYp
Country: Pakistan
Country Code: PK
--------------------------------------------------------------------------------
Text: @ijustine Big Coach Taylor Energy #hispeed #AppleEvent

In [25]:
print(all_tweets)

[{'text': "#AppleEvent\nBasically Apple thinks even if they will sell poop people will buy it 🤷🏻\u200d♂️.\n\nIt's just a 4s with bette… https://t.co/EeAXgwSzCf", 'country': 'Pakistan', 'country_code': 'PK'}, {'text': "#AppleEvent\nI have joke on iphone 12 \nBut go and buy it for me first. \nThen I'll tell u.😏😉\n@Apple #iPhone12", 'country': 'India', 'country_code': 'IN'}, {'text': 'I loved the blue 😍😍\n#AppleEvent\n#Apple12\n#iPhone12', 'country': 'Kingdom of Saudi Arabia', 'country_code': 'SA'}, {'text': "#AppleEvent\nIt's just iphone 4s with a better body but price 🤦🏼\u200d♂️🤔.\n\nWithout charger and earphones 🤦🏼\u200d♂️.\n\nBeyond… https://t.co/zzHwUV6aYp", 'country': 'Pakistan', 'country_code': 'PK'}, {'text': '@ijustine Big Coach Taylor Energy #hispeed #AppleEvent #productRED https://t.co/toTXLU5w8V', 'country': 'United States', 'country_code': 'US'}, {'text': 'hey let’s go back to the iPhone 6 cameras, I’m not really interested in seeing all my wrinkles, thanks #iPhone12 #AppleEv

In [26]:
import numpy as np 
import pandas as pd 
import re  
import nltk # an amazing library to play with natural language
nltk.download('stopwords')  
from nltk.corpus import stopwords 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
tweets = pd.read_csv("https://raw.githubusercontent.com/kolaveridi/kaggle-Twitter-US-Airline-Sentiment-/master/Tweets.csv")
print(tweets)

                 tweet_id  ...               user_timezone
0      570306133677760513  ...  Eastern Time (US & Canada)
1      570301130888122368  ...  Pacific Time (US & Canada)
2      570301083672813571  ...  Central Time (US & Canada)
3      570301031407624196  ...  Pacific Time (US & Canada)
4      570300817074462722  ...  Pacific Time (US & Canada)
...                   ...  ...                         ...
14635  569587686496825344  ...                         NaN
14636  569587371693355008  ...                         NaN
14637  569587242672398336  ...                         NaN
14638  569587188687634433  ...  Eastern Time (US & Canada)
14639  569587140490866689  ...                         NaN

[14640 rows x 15 columns]


In [36]:
X = tweets.iloc[:, 10].values  
y = tweets.iloc[:, 1].values

print(X)

['@VirginAmerica What @dhepburn said.'
 "@VirginAmerica plus you've added commercials to the experience... tacky."
 "@VirginAmerica I didn't today... Must mean I need to take another trip!"
 ... '@AmericanAir Please bring American Airlines to #BlackBerry10'
 "@AmericanAir you have my money, you change my flight, and don't answer your phones! Any other suggestions so I can make my commitment??"
 '@AmericanAir we have 8 ppl so we need 2 know how many seats are on the next flight. Plz put us on standby for 4 people on the next flight?']


In [49]:
#Function for removing web links beggining with http
def remove_words_starting_with(link_text, starting_text):
    while starting_text in link_text: 
        word_start = link_text.find(starting_text)
        if link_text.find(' ', word_start + 1) >=0:  
            word_end = link_text.find(' ', word_start + 1)
        else:
            word_end = len(link_text) + 1
        link_text = link_text[0:word_start] + link_text[word_end:]
    return link_text

#Function for cleaning text
def clean_text(text_in):

  #remove web links
  processed_tweet = remove_words_starting_with(text_in, "http")

  #remove user links
  processed_tweet = remove_words_starting_with(processed_tweet, "@")
      
  # Remove all the special characters
  processed_tweet = re.sub(r'\W', ' ', processed_tweet)

  # remove all single characters
  processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)

  # Remove single characters from the start
  processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 

  # Substituting multiple spaces with single space
  processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)

  # Removing prefixed 'b'
  processed_tweet = re.sub(r'^b\s+', '', processed_tweet)

  # Converting to Lowercase
  processed_tweet = processed_tweet.lower()

  return processed_tweet


In [50]:
processed_tweets = []

for tweet in range(0, len(X)):  
    tweet = clean_text(str(X[tweet]))
    processed_tweets.append(tweet)

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = tfidfconverter.fit_transform(processed_tweets).toarray()


In [52]:
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)  
text_classifier.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [55]:
for tweet in all_tweets:
    print("Unprocessed Tweet: " + str(tweet.get("text")))
    tweet = clean_text(tweet.get("text"))
 
    sentiment = text_classifier.predict(tfidfconverter.transform([tweet]).toarray())
    #print(tweet ,":", sentiment)



Unprocessed Tweet: #AppleEvent
Basically Apple thinks even if they will sell poop people will buy it 🤷🏻‍♂️.

It's just a 4s with bette… https://t.co/EeAXgwSzCf
  processed tweet:  appleevent basically apple thinks even if they will sell poop people will buy it it just 4s with bette 
['negative']
----------------------------------------
Unprocessed Tweet: #AppleEvent
I have joke on iphone 12 
But go and buy it for me first. 
Then I'll tell u.😏😉
@Apple #iPhone12
  processed tweet:  appleevent have joke on iphone 12 but go and buy it for me first then ll tell iphone12
['neutral']
----------------------------------------
Unprocessed Tweet: I loved the blue 😍😍
#AppleEvent
#Apple12
#iPhone12
  processed tweet: i loved the blue appleevent apple12 iphone12
['positive']
----------------------------------------
Unprocessed Tweet: #AppleEvent
It's just iphone 4s with a better body but price 🤦🏼‍♂️🤔.

Without charger and earphones 🤦🏼‍♂️.

Beyond… https://t.co/zzHwUV6aYp
  processed tweet:  appleeve